In [52]:
import re
import numpy as np

with open("data.txt") as file:
    data = file.read()

In [53]:
def fix_workflow(w):

    rules = w[1:-1].split(",")
    end = rules[-1]
    rules = [a.split(":") for a in rules[:-1]]
    temp = {'rules' : rules, "end" : end}

    return temp

workflows, ratings = data.split('\n\n')
ratings = list(map(eval, re.sub(r"(?=[a-z])|(?<=[a-z])",'"', ratings.replace("=", ":")).splitlines()))
workflows = {(split := re.split("(?={)", a))[0] : fix_workflow(split[1]) for a in workflows.splitlines()}

In [54]:
def check_rule(rating, rule):
    letter = rule[0]
    comp = rule[1]
    nr = int(rule[2:])

    if (comp == ">" and rating[letter] > nr) or (comp == "<" and rating[letter] < nr):
        return True
    
    return False

def get_next(key, rating):

    if key in ['A','R']:
        return key

    workflow = workflows[key]

    for rule in workflow['rules']:
        if check_rule(rating, rule[0]):
            return get_next(rule[1], rating)

    return get_next(workflow['end'], rating)

sum([sum(rating.values()) for rating in ratings if get_next("in", rating) == "A"])

397134

In [66]:
def split_rule(rule, ranges):
    letter = rule[0]
    comp = rule[1]
    nr = int(rule[2:])

    rang = ranges[letter]

    if comp == ">":
        r1 = (nr + 1, rang[1])
        r2 = (rang[0], nr)

    elif comp == "<":
        r1 = (rang[0], nr - 1)
        r2 = (nr, rang[1])

    true_range = ranges.copy()
    true_range[letter] = r1

    false_range = ranges.copy()
    false_range[letter] = r2

    return true_range, false_range


def count_possibilites(key, ranges):

    if key == "A":
        return np.prod([a[1] - a[0] + 1 for a in ranges.values()])
    if key == "R":
        return 0

    workflow = workflows[key]

    nr_possibilities = 0

    for rule in workflow['rules']:
        true_range, false_range = split_rule(rule[0], ranges)
        nr_possibilities += count_possibilites(rule[1], true_range)
        ranges = false_range

    nr_possibilities += count_possibilites(workflow["end"], false_range)

    return nr_possibilities


ranges = {
    'x' : (1,4000),
    'm' : (1,4000),
    'a' : (1,4000),
    's' : (1,4000),
}

count_possibilites('in', ranges)

127517902575337